# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import spacy
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from IPython.display import display
from tqdm import notebook
from sklearn.utils import shuffle

import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data = data.sample(50000).reset_index(drop=True)
display(data, data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
text     50000 non-null object
toxic    50000 non-null int64
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


,text,toxic
0,"Cleanup tag\nHi there,\nDo you have any specif...",0
1,"""\nNo, K-Dee did not found that group. I remov...",0
2,This is not a blog. Will both sides please cea...,0
3,added section - Medical Practice \n\nGosnell's...,0
4,"School naming \nAh, I see - Even so, with scho...",0
...,...,...
49995,"Getting around the admins and users \nHi, . I'...",0
49996,"""\nWell, it is an """"occupation"""", just not a g...",0
49997,http://www.telegraph.co.uk/culture/music/35634...,0
49998,SUPER GAY SUPER GAY SUPER GAY SUPER GAY SUPER ...,1


None

In [3]:
data['toxic'].value_counts()

0    44961
1     5039
Name: toxic, dtype: int64

Видно, что пропусков нет, типы данных соответствуют, есть дисбалланс классов.

Далее все комментарии прогоняются через функцию, которая находит леммы слов и очищает комментарии от знаков пунктуации, смайликов:

In [4]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

corpus = data['text'].values.astype('U')
result = []
def lemmatize(text):
    for i in notebook.tqdm(text):
        r = re.sub(r"[^a-zA-Z ]"," ", i)
        lemm_list = (" ".join(r.split()))
        doc = nlp(lemm_list)
        lemm_text = ' '.join(token.lemma_ for token in doc)
        result.append(lemm_text)
    return result

In [5]:
data['clear_text'] = lemmatize(corpus)

Далее разбиваю датасет на выборки тренировочных и тестовых признаков и таргетов:

In [6]:
train, test = train_test_split(data, test_size=0.25, random_state=12345)
tf_idf_train = train['clear_text']
tf_idf_test = test['clear_text']
target_train = train['toxic']
target_test = test['toxic']

Выполняю апсемплинг, чтобы уравновесить классы:

In [7]:
repeat = len(target_train[target_train == 0]) // len(target_train[target_train == 1])
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(tf_idf_train, target_train, repeat)

Расчёт TF-IDF:

In [8]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

tf_idf = TfidfVectorizer(stop_words=stopwords).fit(features_upsampled.values.astype('U'))
features_train = tf_idf.transform(features_upsampled.values.astype('U'))
features_test = tf_idf.transform(tf_idf_test.values.astype('U'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Вывод

В данном блоке я выполнил предобработку данных: лемматизировал текст, очистил от знаков пунктуации, избавился от дисбаланса классов и вычислил TF-IDF.

## Обучение

Обучаю модель логистической регрессии:

In [133]:
model_log = LogisticRegression(random_state=12345, solver='saga', C=1.7, multi_class='multinomial')
model_log.fit(features_train, target_upsampled)
predict = model_log.predict(features_test)
print(f1_score(predict, target_test))

0.7550713749060857


Далее гридсёрчем перебираю параметры для модели дерева решений:

In [120]:
gridParams = {
    'max_depth':range(307,308)
    }

model_tree =  DecisionTreeClassifier(random_state=12345, criterion='entropy', splitter='random', min_samples_split=12)
grid = GridSearchCV(model_tree, gridParams, cv=5, scoring='f1')
grid.fit(features_train, target_upsampled)
print(grid.best_params_)

{'max_depth': 307}


In [121]:
model_tree =  DecisionTreeClassifier(random_state=12345, max_depth=307, criterion='entropy', splitter='random', min_samples_split=12)
model_tree.fit(features_train, target_upsampled)
predict = model_tree.predict(features_test)
print(f1_score(predict, target_test))

0.6787785079242366


Далее перебираю параметры для модели случайного леса:

In [161]:
gridParams = {
    'max_depth':range(476,477),
    'n_estimators':range(8,9)
    }

model_forest =  RandomForestClassifier(random_state=12345, max_features=10000, min_samples_split=3, min_samples_leaf=4)
grid = GridSearchCV(model_forest, gridParams, cv=5, scoring='f1')
grid.fit(features_train, target_upsampled)
print(grid.best_params_)

{'max_depth': 476, 'n_estimators': 8}


In [162]:
model_forest =  RandomForestClassifier(random_state=12345, max_depth=476, n_estimators=8, max_features=10000, min_samples_split=3, min_samples_leaf=4)
model_forest.fit(features_train, target_upsampled)
predict = model_forest.predict(features_test)
print(f1_score(predict, target_test))

0.7091666666666666


## Выводы

Лучшее значение показала модель логистической регрессии, далее идёт случайный лес, и хуже всех - дерево решений.

В этот раз моделью логистической регрессии я достиг необходимого значения метрики, но остальные 2 модели так и не дошли до 0.75. Возможно, если бы не падал кернел и удалось лемматизировать весь датасет, то качество моделей еще бы подросло.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны